# Video Swin Transformers in Pain Detection
## A Comprehensive Evaluation of Effectiveness, Generalizability, and Explainability

This notebook contains the code implementation for the paper *Video Swin Transformers in Pain Detection: A Comprehensive Evaluation of Effectiveness, Generalizability, and Explainability*. We evaluate the effectiveness of Video Swin Transformers (VST) in automated pain detection, highlight their generalizability across datasets, and offer insights into the explainability of model decisions.

## Setup

In [ ]:
# Import Libraries
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, sampler, DataLoader, WeightedRandomSampler
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision.io import read_image
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import video as video_torch
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
import pandas as pd
import cv2
import sys
import csv
from time import time
from itertools import product
import xml.etree.ElementTree as ET
from xml.dom import minidom
from sklearn.metrics import confusion_matrix, classification_report

# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
## CONFIG
LEARNING_RATE = 0.0001
BATCH_SIZE = 16
HYPE_TUNING = True
OPTI = "adam"
WORKERS = 0

In [ ]:
## Non-existing files
no_filenames = ['tv095t2aeunaff001', 'tv095t2aeunaff002','tv095t2aeunaff003','tv095t2aeunaff004','tv095t2aeunaff005','tv095t2aeunaff006','tv095t2aeunaff007']


transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Resize((224,224))                                         
         ]
    )

# 5 folds
la = ["bn080", "mg101", "aa048", "jh043", "jh123"]
lb = ["dn124", "vw121", "fn059", "dr052", "ll042"]
lc = ["jk103", "jl047", "ch092", "jy115", "bg096"]
ld = ["bm049", "kz120", "th108", "tv095", "ib109"]
le = ["nm106", "mg066", "hs107", "gf097", "ak064"]

folds = [la, lb, lc, ld, le]

## Data Loading and Preprocessing
Load the UNBC McMaster and BioVid datasets, and preprocess them for model training.

In [2]:
## Load dataset for UNBC on frame-lvl
class FramesDataset(Dataset):
    def __init__(self, mode, test_groups ,transform=None):
        ## UNBC lables file
        dataset_df = pd.read_csv("<PATH>/modified_label_UNBC.csv")
        # delete non existing files
        frames_to_delete = [i for i in range(1, 8)]
        self.dataset_file = dataset_df[~((dataset_df['participant_id'] == 'tv095') & (dataset_df['video_id'] == 't2aeunaff') & (dataset_df['frame_number'].isin(frames_to_delete)))]

        if mode == "train":
          # put all filesnames from csv which have not the test groups
            self.frames_label_df = self.dataset_file[~self.dataset_file['participant_id'].isin(test_groups)]
            #self.frames_label_df = self.frames_label_df[:20]

        elif mode == "test":
          # put all filesnames from csv which have not the test groups
            self.frames_label_df = self.dataset_file[self.dataset_file['participant_id'].isin(test_groups)]
            #self.frames_label_df = self.frames_label_df[:20]
        else:
            sys.exit("mode not correct")


        self.transform = transform
        self.ps = len(self.frames_label_df[self.frames_label_df['Label'] == 1])
        self.ns = len(self.frames_label_df) - self.ps

        self.frames_label_df['weight'] = self.frames_label_df['Label'].apply(lambda x: self.ps if x == 0 else self.ns)
        self.labels = self.frames_label_df['Label']
        self.weight = self.frames_label_df['weight'].tolist()

    def __len__(self):
        return len(self.frames_label_df)

    def __getitem__(self, idx):
        row = self.frames_label_df.iloc[idx]
        frame_idx = row['frame_number']
        video_name = row['participant_id'] + row['video_id']
        filename = video_name + str(frame_idx).zfill(3)
        frame_path = '<PATH>/Images_Original/' + row['participant_id'] + '/' + video_name + '/' + filename + '.png'
        frame = cv2.imread(frame_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if self.transform:
            frame = transform(frame)
        label = self.frames_label_df.iloc[idx]['Label']

        return frame, label

In [2]:
## Load dataset for UNBC on video-lvl
class VideoFramesDataset(Dataset):
    def __init__(self, mode, test_groups ,transform=None):

        dataset_df = pd.read_csv("<PATH>/modified_label_UNBC.csv")
        # delete non existing files
        frames_to_delete = [i for i in range(1, 8)]
        self.dataset_file = dataset_df[~((dataset_df['participant_id'] == 'tv095') & (dataset_df['video_id'] == 't2aeunaff') & (dataset_df['frame_number'].isin(frames_to_delete)))]

        if mode == "train":
          # put all filesnames from csv which have not the test groups
            self.frames_label_df = self.dataset_file[~self.dataset_file['participant_id'].isin(test_groups)]
            self.frames_label_df = self.frames_label_df[:20]

        elif mode == "test":
          # put all filesnames from csv which have not the test groups
            self.frames_label_df = self.dataset_file[self.dataset_file['participant_id'].isin(test_groups)]
            self.frames_label_df = self.frames_label_df[:20]
        else:
            sys.exit("mode not correct")
        self.transform = transform
        self.ps = len(self.frames_label_df[self.frames_label_df['Label'] == 1])
        self.ns = len(self.frames_label_df) - self.ps

        self.frames_label_df['weight'] = self.frames_label_df['Label'].apply(lambda x: self.ps if x == 0 else self.ns)
        self.labels = self.frames_label_df['Label']
        self.weight = self.frames_label_df['weight'].tolist()

    def __len__(self):
        return len(self.frames_label_df)

    def __getitem__(self, idx):

        row = self.frames_label_df.iloc[idx]
        frame_idx = row['frame_number']
        start_idx = max(1, frame_idx - 3)
        end_idx = frame_idx + 1
        frames = []

        for i in range(start_idx, end_idx):
            video_name = row['participant_id'] + row['video_id']
            filename = video_name + str(i).zfill(3)
            if filename in no_filenames:
                continue
            frame_path = '<PATH>/Images_Original/' + row['participant_id'] + '/' + video_name + '/' + filename + '.png'
            frame = cv2.imread(frame_path)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if self.transform:
                frame = transform(frame)
            frames.append(frame)

        while len(frames) < 4:
            frames.insert(0, frames[0])

        stacked_frames = torch.stack(frames, dim=1)

        label = self.frames_label_df.iloc[idx]['Label']

        return stacked_frames, label


In [2]:
## Load dataset for BioVid on video-lvl
class BioVidTestDataset(Dataset):
    def __init__(self, transform=None):
        self.dataset_path = "<PATH>/BioVid"
        self.transform = transform
        self.subjects = [subject for subject in sorted(os.listdir(self.dataset_path)) if subject != ".DS_Store"]

        self.csv_filename = "<PATH>/dataset_bv.csv"
        self.data = self.read_csv()

    def read_csv(self):
        data = []
        excluded_samples = {'080609_w_27_BL1_98_frame62', '080609_w_27_BL1_98_frame87', '101216_m_40_BL1_100_frame87',
                            '101216_m_40_BL1_100_frame112', '101216_m_40_BL1_100_frame137',
                            '101814_m_58_PA3_14_frame100', '101814_m_58_PA3_14_frame125', '101814_m_58_PA4_22_frame125'}

        with open(self.csv_filename, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            next(csv_reader)  # Skip header row
            for row in csv_reader:
                sample_name, label = row
                if sample_name not in excluded_samples:
                    data.append((sample_name, int(label)))
        return data


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample_name, label = self.data[idx]

        split_sample_name = sample_name.split("_")
        subject_id = f"{split_sample_name[0]}_{split_sample_name[1]}_{split_sample_name[2]}"
        class_id = split_sample_name[3]

        file_path_subject = os.path.join(self.dataset_path, subject_id)
        file_path_class = os.path.join(file_path_subject, class_id)

        # Load and stack the four subframes
        subframes = []
        for subframe_num in range(1, 5):
            subframe_name = f"{sample_name}_subframe_{subframe_num}.jpg"
            subframe_path = os.path.join(file_path_class, subframe_name)
            subframe = Image.open(subframe_path).convert("RGB")
            if self.transform:
                subframe = self.transform(subframe)
            subframes.append(subframe)

        stacked_frames = torch.stack(subframes, dim=1)

        return stacked_frames, label

## Model Architecture

In [3]:
## Build function for VST model architecture
def build_model(unfreezed_layers = 0,  num_classes=2):
    # PRETRAINED WITH KINETICS400_V1
    model = video_torch.swin3d_s(weights='DEFAULT')

    # Freeze all parameters first
    for param in model.parameters():
        param.requires_grad = False

    if unfreezed_layers == 0:
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 2:
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 4:
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5].parameters():
            param.requires_grad = True
        for param in model.features[4][17].parameters():
            param.requires_grad = True
        for param in model.features[4][16].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    # 4 BUT WITHOUT PATCHING MATCH
    if unfreezed_layers == 5:
        for param in model.features[6].parameters():
            param.requires_grad = True
        #for param in model.features[5].parameters():
        #    param.requires_grad = True
        for param in model.features[4][17].parameters():
            param.requires_grad = True
        for param in model.features[4][16].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 6:
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5].parameters():
            param.requires_grad = True
        for param in model.features[4][17].parameters():
            param.requires_grad = True
        for param in model.features[4][16].parameters():
            param.requires_grad = True
        for param in model.features[4][15].parameters():
            param.requires_grad = True
        for param in model.features[4][14].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 8:
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5].parameters():
            param.requires_grad = True
        for param in model.features[4][17].parameters():
            param.requires_grad = True
        for param in model.features[4][16].parameters():
            param.requires_grad = True
        for param in model.features[4][15].parameters():
            param.requires_grad = True
        for param in model.features[4][14].parameters():
            param.requires_grad = True
        for param in model.features[4][13].parameters():
            param.requires_grad = True
        for param in model.features[4][12].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 10:
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5].parameters():
            param.requires_grad = True

        for param in model.features[4][17].parameters():
            param.requires_grad = True
        for param in model.features[4][16].parameters():
            param.requires_grad = True
        for param in model.features[4][15].parameters():
            param.requires_grad = True
        for param in model.features[4][14].parameters():
            param.requires_grad = True
        for param in model.features[4][13].parameters():
            param.requires_grad = True
        for param in model.features[4][12].parameters():
            param.requires_grad = True
        for param in model.features[4][11].parameters():
            param.requires_grad = True
        for param in model.features[4][10].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 99:
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5].parameters():
            param.requires_grad = True
        for param in model.features[4].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True
    model.head = nn.Linear(
        in_features=768,
        out_features= num_classes,
        bias=True
    )
    return model

In [ ]:
## Build function for Swin model (2D) architecture
def build_2Dswin_model(unfreezed_layers=0, num_classes=2):

    # PRETRAINED WITH IMAGENET1K_V1
    model = models.swin_s(weights='DEFAULT')

    # Freeze all parameters first
    for param in model.parameters():
        param.requires_grad = False

    if unfreezed_layers == 0:
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 2:
        for param in model.features[7].parameters():
            param.requires_grad = True
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 4:
        for param in model.features[7].parameters():
            param.requires_grad = True
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5][17].parameters():
            param.requires_grad = True
        for param in model.features[5][16].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 6:
        for param in model.features[7].parameters():
            param.requires_grad = True
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5][17].parameters():
            param.requires_grad = True
        for param in model.features[5][16].parameters():
            param.requires_grad = True
        for param in model.features[5][15].parameters():
            param.requires_grad = True
        for param in model.features[5][14].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True

    if unfreezed_layers == 8:
        for param in model.features[7].parameters():
            param.requires_grad = True
        for param in model.features[6].parameters():
            param.requires_grad = True
        for param in model.features[5][17].parameters():
            param.requires_grad = True
        for param in model.features[5][16].parameters():
            param.requires_grad = True
        for param in model.features[5][15].parameters():
            param.requires_grad = True
        for param in model.features[5][14].parameters():
            param.requires_grad = True
        for param in model.features[5][13].parameters():
            param.requires_grad = True
        for param in model.features[5][12].parameters():
            param.requires_grad = True
        for param in model.head.parameters():
            param.requires_grad = True


    model.head = nn.Linear(
        in_features=768,
        out_features=num_classes,
        bias=True
    )
    return model

In [ ]:
## Build function for ViT model (2D) architecture
def build_model_vit(unfreezed_layers = 0,  num_classes=2):
    # PRETRAINED WITH IMAGENET1K_V1
    model = (models.vit_b_16(weights='DEFAULT'))

    model.heads = nn.Linear(
        in_features=768,
        out_features=num_classes,
        bias=True
    )

    # Freeze all parameters first
    for param in model.parameters():
        param.requires_grad = False

    for param in model.heads.parameters():
        param.requires_grad = True

    if unfreezed_layers == 0:
        pass

    if unfreezed_layers == 2:
        for param in model.encoder.layers[11].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[10].parameters():
            param.requires_grad = True

    if unfreezed_layers == 4:
        for param in model.encoder.layers[11].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[10].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[9].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[8].parameters():
            param.requires_grad = True

    if unfreezed_layers == 6:
        for param in model.encoder.layers[11].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[10].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[9].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[8].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[7].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[6].parameters():
            param.requires_grad = True

    if unfreezed_layers == 8:
        for param in model.encoder.layers[11].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[10].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[9].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[8].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[7].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[6].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[5].parameters():
            param.requires_grad = True
        for param in model.encoder.layers[4].parameters():
            param.requires_grad = True


    return model

## Training functions for a single fold 

In [4]:
def model_training_single_fold(fold, top, training_optimizer, learning_rate, batches_size, layers, wd, model_type, device):

    # CEL as loss function
    criterion = nn.CrossEntropyLoss()

    # Focal Loss 
    # criterion = kornia.losses.focal_loss(pred=outputs, target=labels, alpha=alpha, gamma=gamma, reduction="mean")

    # Load datasets
    if model_type == "vst_3d":
        model = build_model(unfreezed_layers=layers, num_classes=2)
        train_ds = VideoFramesDataset("train", fold, 4, transform)
        test_ds = VideoFramesDataset("test", fold, 4, transform)
    else:
        train_ds = FramesDataset("train", fold, transform)
        test_ds = FramesDataset("test", fold, transform)
        if model_type == "swin_2d":
            model = build_2Dswin_model(unfreezed_layers=layers, num_classes=2)
        elif model_type == "vit":
            model = build_model_vit(unfreezed_layers=layers, num_classes=2)

    model.to(device)
    # pain class is over-sampled to prevent overfitting on the majority class
    weight_sampler = WeightedRandomSampler(weights=train_ds.weight, num_samples=len(train_ds), replacement=True)

    # set up the data loader
    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batches_size, num_workers=WORKERS, sampler=weight_sampler, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batches_size, num_workers=WORKERS, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=wd)

    all_epoch_results = []
    # training start
    print(f"Model type:{model_type}, Opt: {training_optimizer}, learning_rate: {learning_rate}, batch_size: {batches_size}, Unfreezed layers (except head): {layers}, weight decay:{wd}")
    for epoch in range(NUM_EPOCHS):
        start = time()
        model.train()
        running_loss = 0.0
        for b, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.long()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        # Evaluation
        f1, auc = evaluation(model, test_loader, device)
        all_epoch_results.append({
            "epoch": epoch + 1,
            "f1": f1,
            "auc": auc
        })
        print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {running_loss / len(train_loader)}, F1: {f1}, AUC: {auc}")
        end = time()
        time_epoch = end - start
        print("Finish " + str(epoch + 1) + " in " + str(time_epoch) + "sec.")

    f = 99
    if fold == ["bn080", "mg101", "aa048", "jh043", "jh123"]:
        f = 1
    if fold == ["dn124", "vw121", "fn059", "dr052", "ll042"]:
        f = 2
    if fold == ["jk103", "jl047", "ch092", "jy115", "bg096"]:
        f = 3
    if fold == ["bm049", "kz120", "th108", "tv095", "ib109"]:
        f = 4
    if fold == ["nm106", "mg066", "hs107", "gf097", "ak064"]:
        f = 5

    ### Save results ###

    return f1, auc

## Hyperparameter Tuning/Optuna

In [ ]:
def hyperparameter_optuna_search_vst(trial, inner_fold, outer_k, temporal_depth):

    # Define hyperparameter space
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    unfreezed_layers = trial.suggest_int("unfreezed_layers", 0, 8, step=2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])
    gamme = trial.suggest_float("gamma", 1, 3, step= 0.5)
    alpha = trial.suggest_float("alpha", 0.6, 0.95, step=0.05)
    loss_fct = "focal_loss"

    if loss_fct == "focal_loss":
        criterion = kornia.losses
    elif loss_fct == "cross_entropy_loss":
        criterion = nn.CrossEntropyLoss()
    else:
        raise ValueError("Unsupported optimizer")

    sum_f1 = 0.0

    for j, inner_k in enumerate(inner_fold):
        start = time()

        model = build_model(unfreezed_layers=unfreezed_layers, num_classes=2)
        model.to(device)

        k_train_exclude = inner_k + outer_k

        train_ds = VideoFramesDataset("train", k_train_exclude, temporal_depth, transform)
        test_ds = VideoFramesDataset("test", inner_k, temporal_depth, transform)

        if loss_fct == "focal_loss":
            weight_sampler = None
        else:
            # pain class is over-sampled to prevent overfitting on the majority class
            weight_sampler = WeightedRandomSampler(weights=train_ds.weight, num_samples=len(train_ds), replacement=True)
        train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, num_workers=WORKERS, sampler=weight_sampler, shuffle=False)
        test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, num_workers=WORKERS, shuffle=False)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        for epoch in range(1):
            model.train()
            for b, data in enumerate(train_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                labels = labels.long()
                optimizer.zero_grad()
                outputs = model(inputs)
                if loss_fct == "focal_loss":
                    loss = criterion.focal_loss(pred=outputs, target=labels, alpha=alpha, gamma=gamme, reduction="mean")
                    #if b % 100 == 0 and b != 0:
                    #    print(b, " loss: ", loss.item())
                else:
                    loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        f1, _ = evaluation(model, test_loader, device)
        sum_f1 += f1

        end = time()
        diff = end - start

        print("Time for one epoch: " + str(diff))

    avg_f1 = sum_f1/4
    return avg_f1

In [ ]:
def training_nested_cv():

    # Loop over all 5 folds and do for each a hyperparameter search
    for i, outer in enumerate(folds):
        print("Fold " + str(i+1))
        inner_folds = [fold for fold in folds if fold != outer]
        study = optuna.create_study(study_name=f'Baseline VST Focal Loss Optimization Fold {i+1}', direction='maximize',sampler=optuna.samplers.TPESampler())
        # Run hyperparameter study
        study.optimize(lambda trial: hyperparameter_optuna_search_vst(trial, inner_folds, outer, 4), n_trials=20, n_jobs=1)
        # Print best score achieved during conducted hyperparameter study
        print('Best Score: ', study.best_trial.value)
        # Print best hyperparameter configuration that were used for obtaining the best value during hyperparameter study
        print("Best Params: ")
        for hyperparameter, value in study.best_trial.params.items():
            print("  {}: {}".format(hyperparameter, value))
        joblib.dump(study, f'<PATH>/optuna_study_batch_fold{i + 1}_vst_focal_loss.pkl')

    return None

## Evaluation

In [5]:
# Evaluation function (F1, AUC)
def evaluation(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    all_pos_output = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            pos_out = outputs[:,1]
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_pos_output.extend(pos_out.cpu().numpy())

    f1_pos_0 = f1_score(all_labels, all_preds, pos_label=1)
    try:
        auc = roc_auc_score(all_labels, all_pos_output)
    except:
        auc = 9999

    return f1_pos_0, auc